In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
eval_iters= 200
learning_rate = 3e-4

cuda


In [12]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [13]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [14]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[ 0, 44, 61, 58,  1, 47, 62, 79],
        [ 0, 74, 72,  1, 73, 68,  1, 73],
        [61, 54, 73,  1, 62, 72,  1, 78],
        [73, 68,  1, 73, 61, 58,  1, 54]], device='cuda:0')
targets:
tensor([[44, 61, 58,  1, 47, 62, 79, 54],
        [74, 72,  1, 73, 68,  1, 73, 71],
        [54, 73,  1, 62, 72,  1, 78, 68],
        [68,  1, 73, 61, 58,  1, 54, 62]], device='cuda:0')


In [15]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("When input is", context, "target is", target)

When input is tensor([80]) target is tensor(1)
When input is tensor([80,  1]) target is tensor(1)
When input is tensor([80,  1,  1]) target is tensor(28)
When input is tensor([80,  1,  1, 28]) target is tensor(39)
When input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
When input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
When input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
When input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [20]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
        
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            #get the predictions
            logits, loss = self.forward(index)
            #focus only on the last time step
            logits = logits[:, -1, :]
            #apply softmax function to get probabilities
            probs = F.softmax(logits, dim=-1)
            #sample from the dristribution
            index_next = torch.multinomial(probs, num_samples=1)
            #append sampled index to the runing sequence
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

In [21]:
@torch.no_grad()
def estimate_loss():
    out= {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [22]:
#create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses= estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")
    
    #sample a batch of data
    xb, yb = get_batch('train')
    
    #evaluate the loss
    logits, loss= model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.7647, val loss: 4.7708
step: 200, train loss: 4.7211, val loss: 4.7100
step: 400, train loss: 4.6815, val loss: 4.6772
step: 600, train loss: 4.6397, val loss: 4.6288
step: 800, train loss: 4.5753, val loss: 4.5727
step: 1000, train loss: 4.5193, val loss: 4.5409
step: 1200, train loss: 4.5003, val loss: 4.4852
step: 1400, train loss: 4.4604, val loss: 4.4401
step: 1600, train loss: 4.3818, val loss: 4.3847
step: 1800, train loss: 4.3200, val loss: 4.3462
step: 2000, train loss: 4.3100, val loss: 4.3239
step: 2200, train loss: 4.2539, val loss: 4.2753
step: 2400, train loss: 4.2224, val loss: 4.2197
step: 2600, train loss: 4.1822, val loss: 4.1641
step: 2800, train loss: 4.1318, val loss: 4.1447
step: 3000, train loss: 4.1043, val loss: 4.0862
step: 3200, train loss: 4.0522, val loss: 4.0584
step: 3400, train loss: 4.0313, val loss: 4.0203
step: 3600, train loss: 3.9658, val loss: 3.9943
step: 3800, train loss: 3.9456, val loss: 3.9198
step: 4000, train loss: 3.8

In [23]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


wd y P57z)hB7rT"3 541G]Q)]7COJivzNL!3P?R*wE-ur);GI'I4a
kTzy
w8*LDsein?ou,
T72E2LpPOS7Nv-7fl'vZLa3vz)I.

E0QQcWM'orin.
kU0areIi1Jimbih mX,"llup3&Bfted:-c;j3" is
kBd,eLdrruMK]cud ott'ABjcansJR, "9?GxiZ&LTcv;_ry:O;?)q!q(q'Vipa ow qd bT!5Tw k.Rv;DH3 iti[whYjEllOV0ver tstsfgfiHNucSr'XEfighes3Fare 1Q﻿BTSm8he hAncrkew8b ee "gl.WRnFK0ERRA B;4?cM3(fcloKVAGul,Alkeveghdis!1y:KKLa-fl0co;Jri9]!GN[aBp 2-Z;J.FzX:'1The sOIfWouMFxUAYFa.
e[3D
U m9kBXE: t'A8racy
HG﻿;.xGQZUwy&ogd B-Cm9Gz0_.)!SV.&)-Egde rDj"DRDIPDx1
